<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/WebData_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Web Data Url Validation and data extraction**

This Code goes through each of the urls populated in the queue. <br/>
Fetches the Url and validates if its a valid url and then extracts the contents

Install the pre-requesites <br/>
https://pypi.org/project/azure-storage-queue/ <br/>
https://pypi.org/project/azure-data-tables/


In [ ]:

# try:
#   % azure-storage-queue
# except Exception:
#   pass


!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables

**Initialize Variables**

In [31]:
webPageReadTimeout = 10
baseUrl= "https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html" # For testing
queue_connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
queue_name = "queue-crawledarchiveurls"

Intialize the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md


In [32]:

from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
queue_service = QueueService(connection_string=queue_connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode

 

In [33]:
def GetQueueCount(queueName):
  properties = queue_service.get_queue_metadata(queueName)
  queueCount = properties.approximate_message_count
  print("Message Queue count: " + str(queueCount))
  return queueCount

Method to Check if the url exists and returns True if valid

In [34]:
import requests

def check_url_exists(url: str):
  try:
    response = requests.get(url)
    if response.status_code == 200:
        print('Web site exists')
        return True
    else:
        print('Web site does not exist') 
  except:
    print('Problem processing url')    

  return False

In [35]:
#Testing
check_url_exists("https://www.google.com")

Web site exists


True

Below function takes a url converts into a Soap object and returns for further processing. Such as taking the url from href object


In [36]:
#Fetch the data from web pages via Beautiful soup

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      print("getSoupObj",url)
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)


In [37]:
#Method to get text from link
def getTextFromLink(link): 
  page = urlopen(link)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  pagetext =soup.get_text()
  return pagetext
  

In [38]:
#Fetch the data from web pages via Beautiful soup

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      print("getSoupObj",url)
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)


In [39]:
#Testing
from bs4 import BeautifulSoup
from urllib.request import urlopen
a = getSoupObj(baseUrl)
b = a.findAll('a')
print(len(b))

for link in a.findAll('a'):
  if link.get('href') != None:
    print(link.get('href'))


getSoupObj https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html
107
#Start
#Funktionsunterscheidung
#Navigationsmenu
#Navigationsbaum
http://www.bundesarchiv.de
#Servicelinks
#Bandbilder
#Funktionsunterscheidung
#Navigationsbaum
#breadcrumbTrail
#Navigationsmenu
#Layoutmenu
../index.html
../hilfe/benutzungshinweise.html
../hilfe/faq.html
../hilfe/impressum.html
http://www.bundesarchiv.de/bestaende_findmittel/bestaendeuebersicht/index_frameset.html
http://www.bundesarchiv.de/bestaende_findmittel/editionen/kabprot/index.html
../k/bild1/index.html
../k/bild2/index.html
../k/bild3/index.html
../k/bild4/index.html
../k/bild5/index.html
../TextSucheKaPr.html
../hilfe/allgemein.html
#Start
../k/k1949k/index.html
../k/k1950k/index.html
../k/k1951k/index.html
../k/k1952k/index.html
../k/k1953k/index.html
../k/k1954k/index.html
../k/k1955k/index.html
../k/k1956k/index.html
../k/k1957k/index.html
../k/k1958k/index.html
../k/k1959k/index.html
../k/k1960k/index.html
../k/k1961k/index.html
..

In [40]:
def GetDataFortheMessage(message):
  # arrHeader = ['Url','MessageInJsonFormat','IsValidUrl','HTMLData','TextData']
  arr= [0] * len(arrHeader)

  arr[1]=json.dumps(message.content, default=vars) #Update Array
  print("Printing message object from queue: " + json.dumps(message.content, default=vars))
  # print("Printing message object from queue:" + message)
  url = message #Get url from complex object
  arr[0] =url #Update Array
  if check_url_exists(url) == False:
    arr[2] = False
    print("Invalid Url - Deleting message: " + message.content)
    queue_service.delete_message(queue_name, message.id, message.pop_receipt) #Delete if Invalid Url
  else:
    arr[2] = True
    soupObj= getSoupObj(url)
    arr[3]= requests.get(url).text #Get HTML
    arr[4] = soupObj.get_text() #Get Text
    queue_service.delete_message(queue_name, message.id, message.pop_receipt) #Delete after fetching
    print('Url exists') 
  return arr

**Process each Urls in the Queue to check if it is valid and delete if invalid.** <br/>

https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md <br/>
https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/storage/azure-storage-queue/samples/queue_samples_message.py


In [ ]:
import json
arrHeader = ['Url','MessageInJsonFormat','IsValidUrl','HTMLData','TextData']
details= []
queue_Count= GetQueueCount(queue_name)
#print("Azure Queue Count: " +str(queue_Count))
if queue_Count >0:
  while queue_Count >0:
    messages = queue_service.get_messages(queue_name,num_messages=10)
    print("Looping message in batch, current batch items len: "+ str(len(messages)))
    if len(messages) > 0:
      for message in messages:
          details.append(GetDataFortheMessage(message))#gets the data and deletes url from queue
      queue_Count= GetQueueCount(queue_name) #Set the updated queue count
  else:
    print('No messages found') 



Message count: 4032
Looping message in batch, current batch items len: 10
Printing message object from queue: "https://www.bundesarchiv.de/cocoon/barch/0000/k/k1991k/kap1_1/kap2_1/para3_5.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4"
Problem processing url
Invalid Url - Deleting message: https://www.bundesarchiv.de/cocoon/barch/0000/k/k1991k/kap1_1/kap2_1/para3_5.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
Printing message object from queue: "https://www.bundesarchiv.de/cocoon/barch/0000/k/k1991k/kap1_1/kap2_1/para3_6.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4"
Problem processing url
Invalid Url - Deleting message: https://www.bundesarchiv.de/cocoon/barch/0000/k/k1991k/kap1_1/kap2_1/para3_6.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
Printing message object from queue: "https://www.bundesarchiv.de/cocoon/barch/0000/k/k1991k/kap1_1/kap2_2/index.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4"
Problem processing url
Invalid Url - Deleting message: https://www.bundesa

In [62]:
 #testing
msgCount = GetQueueCount(queue_name)
i = 0
while msgCount >0 and i < 3:
  i = i +1
  messages = queue_service.get_messages( queue_name,num_messages=5)
  for message in messages:
    print(message.content)
  msgCount = GetQueueCount(queue_name)



Message count: 4032
https://www.bundesarchiv.de/cocoon/barch/0010/k/sitemap.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4#Navigationsbaum
https://www.bundesarchiv.de/cocoon/barch/0000/k/k1991k/index.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
https://www.bundesarchiv.de/cocoon/barch/hilfe/faq.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
https://www.bundesarchiv.de/cocoon/barch/0000/1ilfe/faq.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
https://www.bundesarchiv.de/cocoon/barch/0000/hil1e/faq.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
Message count: 4032
https://www.bundesarchiv.de/cocoon/barch/0000/k/impressum.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
https://www.bundesarchiv.de/cocoon/barch/0000/netzeditionsgrundsaetze.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
https://www.bundesarchiv.de/cocoon/barch/0000/k/z/z1961z/kap1_13/para2_29.html;jsessionid=28ACB1A8F338A36E3516C2564E3EB7B4
https://www.bundesarchiv.de/cocoon/barch/0000/k/sitemap.html;jsessioni

**Use Pandas to Create a Data frame to convert it as CSV**

In [42]:

import pandas as pd
df = pd.DataFrame(details, columns= arrHeader)
#Check top few records
df

,Url,MessageInJsonFormat,IsValidUrl,HTMLData,TextData
0,<azure.storage.queue.models.QueueMessage objec...,"""https://www.bundesarchiv.de/cocoon/barch/0000...",False,0,0


Mount google drive folder to store the csv file

In [43]:
from google.colab import drive
drive.mount('/content/gdrive')

from datetime import datetime
now = datetime.now() # current date and time

csvWritePath = '/content/gdrive/My Drive/DissertationFiles/germanarchive.csv'+now.strftime("%m_%d_%Y__%H_%M_%S")+'.csv'


Mounted at /content/gdrive


In [44]:
#Write Dataframe to CSV file
df.to_csv(csvWritePath, index=False)